In [1]:
# Setup: Connect to PostgreSQL database
import os
import psycopg2
import pandas as pd
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Connect to PostgreSQL
conn = psycopg2.connect(
    host=os.getenv("DB_HOST", "localhost"),
    port=os.getenv("DB_PORT", "5433"),
    user=os.getenv("DB_USER", "postgres"),
    password=os.getenv("DB_PASSWORD"),
    database=os.getenv("DB_NAME", "students_mental_health_db")
)

print("✅ Successfully connected to PostgreSQL database!")

✅ Successfully connected to PostgreSQL database!


![Illustration of silhouetted heads](mentalhealth.jpg)

Does going to university in a different country affect your mental health? A Japanese international university surveyed its students in 2018 and published a study the following year that was approved by several ethical and regulatory boards.

The study found that international students have a higher risk of mental health difficulties than the general population, and that social connectedness (belonging to a social group) and acculturative stress (stress associated with joining a new culture) are predictive of depression.


Explore the `students` data using PostgreSQL to find out if you would come to a similar conclusion for international students and see if the length of stay is a contributing factor.

Here is a data description of the columns you may find helpful.

| Field Name    | Description                                      |
| ------------- | ------------------------------------------------ |
| `inter_dom`     | Types of students (international or domestic)   |
| `japanese_cate` | Japanese language proficiency                    |
| `english_cate`  | English language proficiency                     |
| `academic`      | Current academic level (undergraduate or graduate) |
| `age`           | Current age of student                           |
| `stay`          | Current length of stay in years                  |
| `todep`         | Total score of depression (PHQ-9 test)           |
| `tosc`          | Total score of social connectedness (SCS test)   |
| `toas`          | Total score of acculturative stress (ASISS test) |

In [2]:
# Explore the students data
query = """
SELECT inter_dom, japanese_cate, english_cate, academic, age, stay, todep, tosc, toas
FROM students
LIMIT 9;
"""

df_preview = pd.read_sql_query(query, conn)
display(df_preview)

C:\Users\Asus\AppData\Local\Temp\ipykernel_12616\3862356994.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_preview = pd.read_sql_query(query, conn)


,inter_dom,japanese_cate,english_cate,academic,age,stay,todep,tosc,toas
0,Inter,Average,High,Grad,24.0,5.0,0.0,34.0,91.0
1,Inter,High,High,Grad,28.0,1.0,2.0,48.0,39.0
2,Inter,High,High,Grad,25.0,6.0,2.0,41.0,51.0
3,Inter,Low,Average,Grad,29.0,1.0,3.0,37.0,75.0
4,Inter,Low,Average,Grad,28.0,1.0,3.0,37.0,82.0
5,Inter,Average,High,Grad,24.0,6.0,6.0,38.0,83.0
6,Inter,Average,High,Grad,23.0,1.0,3.0,46.0,58.0
7,Inter,Low,Low,Grad,30.0,2.0,9.0,41.0,127.0
8,Inter,High,High,Grad,25.0,4.0,7.0,36.0,109.0


In [4]:
# Main Analysis: Impact of length of stay on mental health for international students
query = """
SELECT 
	stay,
	COUNT(inter_dom) AS count_int,
	ROUND(CAST(AVG(todep) AS numeric), 2) AS average_phq,
	ROUND(CAST(AVG(tosc) AS numeric), 2) AS average_scs,
	ROUND(CAST(AVG(toas) AS numeric), 2) AS average_as
FROM students
WHERE inter_dom = 'Inter'
GROUP BY stay	
ORDER BY stay DESC;
"""

df_result = pd.read_sql_query(query, conn)
display(df_result)

# Interpretation
print("\n📊 Key Findings:")
print("- PHQ-9 (average_phq): Depression score (higher = more depressive symptoms)")
print("- SCS (average_scs): Social connectedness (higher = better social connections)")
print("- ASISS (average_as): Acculturative stress (higher = more stress adapting to new culture)")

C:\Users\Asus\AppData\Local\Temp\ipykernel_12616\2170181120.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = pd.read_sql_query(query, conn)


,stay,count_int,average_phq,average_scs,average_as
0,10.0,1,13.00,32.00,50.00
1,8.0,1,10.00,44.00,65.00
2,7.0,1,4.00,48.00,45.00
3,6.0,3,6.00,38.00,58.67
4,5.0,1,0.00,34.00,91.00
5,4.0,14,8.57,33.93,87.71
6,3.0,46,9.09,37.13,78.00
7,2.0,39,8.28,37.08,77.67
8,1.0,95,7.48,38.11,72.80



📊 Key Findings:
- PHQ-9 (average_phq): Depression score (higher = more depressive symptoms)
- SCS (average_scs): Social connectedness (higher = better social connections)
- ASISS (average_as): Acculturative stress (higher = more stress adapting to new culture)
